In [1]:
%%bash
OUTDIR="./data/real_estate/"
OUTFILE="real_estate_valuation.xlsx"

if [ ! -d $OUTDIR ]; then
    mkdir $OUTDIR
fi

if [ ! -f $OUTDIR/$OUTFILE ]; then
    cd $OUTDIR
    curl -o $OUTFILE "https://archive.ics.uci.edu/ml/machine-learning-databases/00477/Real%20estate%20valuation%20data%20set.xlsx"

fi

# Libraries

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

# Setup

In [3]:
seed = 83282168
np.random.seed(seed)

In [4]:
datadir = Path("./data/real_estate")

In [5]:
df = pd.read_excel(datadir / "real_estate_valuation.xlsx")

In [6]:
df.columns = ['id', 'transaction_date', 'house_age', 'nearest_subway_m', 'n_conv_store', 'latitude', 'longitude', 'unit_price']

In [7]:
df.head(10)

,id,transaction_date,house_age,nearest_subway_m,n_conv_store,latitude,longitude,unit_price
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1
5,6,2012.666667,7.1,2175.03000,3,24.96305,121.51254,32.1
6,7,2012.666667,34.5,623.47310,7,24.97933,121.53642,40.3
7,8,2013.416667,20.3,287.60250,6,24.98042,121.54228,46.7
8,9,2013.500000,31.7,5512.03800,1,24.95095,121.48458,18.8
9,10,2013.416667,17.9,1783.18000,3,24.96731,121.51486,22.1


# Splitting the Data

In [8]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [9]:
df['month'] = round((df.transaction_date - 2012) * 12).astype(int)

In [10]:
df['year'] = 2012 + (df.month > 12)
df['month'] = df.month - (df.month > 12) * 12

In [11]:
train, test = train_test_split(df, test_size = 0.3, stratify = df[['year','month']])

# Data Preparation

In [12]:
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

from lib.custom_transforms import DtypeMapper, DropColumn, TransformByDtype, PdDummyEncoder

In [13]:
def generate_features(X, copy=False):
    if copy:
        X = X.copy()

    X['inv_house_age'] = 1 / (X['house_age'] + 1)
    X['inv_nearest_subway'] = 1 / (X['nearest_subway_m'] + 1)
    X['inv_house_age_nearest_subway'] = 1 / (X['nearest_subway_m'] * X['house_age'] + 1)

    X['n_conv_utility'] = np.log(X['n_conv_store'] + 1)

    X['quarter'] = np.select(
        condlist = [X.month <= 3, X.month <= 6, X.month <= 9, X.month <= 12],
        choicelist = [1,2,3,4],
        default = -1
    )

    return X

In [14]:
feature_gen = Pipeline([ 
    ('new_features', FunctionTransformer(generate_features, kw_args={'copy' : True})),
    ('set_dtypes', DtypeMapper({'category': ['quarter']})),
    ('drop_col', DropColumn(
        ['id', 'transaction_date', 'house_age', 'nearest_subway_m', 'n_conv_store', 
        'inv_house_age_nearest_subway', 'month'])),
    ('minmax_normalise', TransformByDtype(
        transformer = MinMaxScaler(), 
        include_dtypes = ['number'],
        combine_strategy = 'reassign')),
    ('dummy_encoding', PdDummyEncoder(dummy_na=False, drop_first=True)),

])

In [15]:
train_cp = feature_gen.fit_transform(train)

In [16]:
import statsmodels.api as sm

In [17]:
model = sm.OLS(train_cp['unit_price'], train_cp.loc[:, ~train_cp.columns.isin(['unit_price'])])
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             unit_price   R-squared (uncentered):                   0.932
Model:                            OLS   Adj. R-squared (uncentered):              0.930
Method:                 Least Squares   F-statistic:                              426.8
Date:                Sat, 15 Aug 2020   Prob (F-statistic):                   7.18e-158
Time:                        21:56:24   Log-Likelihood:                          321.99
No. Observations:                 289   AIC:                                     -626.0
Df Residuals:                     280   BIC:                                     -593.0
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
latitude               0.2542      0.037      6.838      0.000       0.181       0.327
longitude              0.1375      0.029      4.665      0.000       0.079       0.196
year                   0.0039      0.016      0.242      0.809      -0.028       0.036
inv_house_age          0.1547      0.025      6.295      0.000       0.106       0.203
inv_nearest_subway     0.2825      0.049      5.796      0.000       0.187       0.378
n_conv_utility         0.0828      0.020      4.162      0.000       0.044       0.122
quarter_2              0.0001      0.012      0.011      0.991      -0.024       0.024
quarter_3             -0.0307      0.016     -1.876      0.062      -0.063       0.002
quarter_4             -0.0224      0.018     -1.269      0.205      -0.057       0.012
==============================================================================
Omnibus:                      219.718   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6892.278
Skew:                           2.657   Prob(JB):                         0.00
Kurtosis:                      26.326   Cond. No.                         14.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Test Set Prepation

In [18]:
test_cp = feature_gen.transform(test)

In [19]:
prediction = model.predict(test_cp.loc[:, ~test_cp.columns.isin(['unit_price'])])

# Model Performance

In [20]:
def MSE(Y_hat, Y_actual):
    return np.mean(np.square(Y_hat - Y_actual))

In [21]:
MSE(prediction, test_cp['unit_price'])

0.005670468127655128